# Scotland Local Area District - Notebook

In [1]:
import pandas as pd
import numpy as np
import zipfile
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
import datetime
import urllib.request, urllib.error
import os.path

## 1. Covid-19 Death Total by Local Authority

In [2]:
all_deaths_path = "data/sco_covid_data/custom_data/all_deaths_lad_18.csv"
covid_deaths_path = "data/sco_covid_data/custom_data/covid_deaths_lad_18.csv"
loc_deaths_all_path = "data/sco_covid_data/custom_data/deaths_by_location_all_lad_18.csv"
loc_deaths_corona_path = "data/sco_covid_data/custom_data/deaths_by_location_corona_lad_18.csv"
path = "https://www.nrscotland.gov.uk/files//statistics/covid19/"

In [3]:
def get_week_num():
    return datetime.date.today().isocalendar()[1]

In [4]:
get_week_num()

20

In [29]:
'''NEED TO REFACTOR THIS'''
def get_covid_deaths_df(path):
            
    # Try get get the most recent data
    try:
        url = path + "covid-deaths-data-week-%d.zip" % (get_week_num()-1)
        conn = urllib.request.urlopen(url)
        zipfile = ZipFile(BytesIO(conn.read()))
        
        df_covid_deaths = pd.read_csv(zipfile.open('covid-deaths-data-week-18_Table 1 - COVID deaths.csv'),header = 3, encoding='unicode-escape')
        df_all_deaths = pd.read_csv(zipfile.open('covid-deaths-data-week-18_Table 2 - All deaths.csv'),header = 3, encoding='unicode-escape')
        
        df_covid_deaths_trimmed = df_covid_deaths.iloc[48:80, 1:get_week_num()+1]
        df_all_deaths_trimmed = df_all_deaths.iloc[50:82, 1:get_week_num()+1]

        
        return df_all_deaths_trimmed, df_covid_deaths_trimmed
    
    # If error, then get last week response
    except urllib.error.HTTPError as e:
        url = path + "covid-deaths-data-week-%d.zip" % (get_week_num()-2)
        conn = urllib.request.urlopen(url)
        zipfile = ZipFile(BytesIO(conn.read()))
        
        df_covid_deaths = pd.read_csv(zipfile.open('covid-deaths-data-week-18_Table 1 - COVID deaths.csv'),header = 3, encoding='unicode-escape')
        
        idx_start = np.where(df_covid_deaths['Unnamed: 1'] == 'Aberdeen City')[0][0]
        idx_end = idx_start + 32
        df_covid_deaths_trimmed = df_covid_deaths.iloc[idx_start:idx_end,1:get_week_num()]
        df_covid_deaths_trimmed = df_covid_deaths_trimmed.set_index('Unnamed: 1')
        
        return df_covid_deaths_trimmed

In [35]:
df_covid_deaths_per_week = get_covid_deaths_df(path).head()

In [50]:
def get_total_deaths_for_lad(df_covid_deaths_per_week, lad_name):
    
    df_lad = df_covid_deaths_per_week.loc[[lad_name]].transpose()
    df_lad = df_lad.reset_index(inplace=True, drop = False)
    df_lad = df_lad.rename(columns={'index':'date', la})
    return df_lad

In [51]:
get_total_deaths_for_lad(df_covid_deaths_per_week, 'Aberdeen City')

Unnamed: 1,index,Aberdeen City
0,30-Dec-19,0.0
1,6-Jan-20,0.0
2,13-Jan-20,0.0
3,20-Jan-20,0.0
4,27-Jan-20,0.0
5,3-Feb-20,0.0
6,10-Feb-20,0.0
7,17-Feb-20,0.0
8,24-Feb-20,0.0
9,2-Mar-20,0.0
